<font size="20">

# LET measurements, April 2020

<font size="3">
    
> __Author:__ A. Backis
<br/>__Institute:__ European Spallation Source (ESS), University of Glasgow (UoG)
<br/>__Date:__ 4/3-2020

_Abstract:_
This notebook contains the data-analysis tools used for the measurements at the LET instrument at ISIS. It describes how the analysis was performed, and summarizes the results.

# Contents
    
* [1. Introduction](#INTRODUCTION)
    * [1.1 Packages](#PACKAGES)
    * [1.2 Global parameters](#PACKAGES)
* [2. Data](#DATA)
    * [2.1 Descriptions](#EXTRACT)
    * [2.2 Paths](#LOAD)
    * [2.3 Extract](#FILTER)
    * [2.4 Load](#FILTER)
    * [2.5 Filter](#FILTER)
    * [2.6 Plotting](#DATA)
* [4. Utgård](#DATA)
* [Acknowledgements](#ACKNOWLEDGEMENTS)
* [References](#REFERENCES)

# 1. Introduction

This notebook summarizes the details of the measurements performed at the LET instrument, at ISIS, England. The notebook contains all details on the preparations leading up to the beam time on the 28:th and 29:th April 2020. This includes preparations at Utgård and initial background testing at ISIS.

## 1.1 Packages<a class="anchor" id="PACKAGES"></a>

In [ ]:
# Autoload packages when doing an external change
%load_ext autoreload
%autoreload 2

# General packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# Data handling
import mg.file_handling.read as mg_read

# Plotting
import mg_let.basic_plot as mg_basic_plot

# Helper functions
import mg.helper_functions.misc as mg_hf

# Make matplotlib interactive mode in notebook
%matplotlib inline
%matplotlib notebook

## 1.2 Global parameters

In [ ]:
US_TO_TDC = 1 / (62.5e-9 * 1e6)
H_TO_TDC = 1 / ((62.5e-9) * 60*60)
RAW_FOLDER = '../data/let/raw/'
PROCESSED_FOLDER = '../data/let/processed/'

# 2. Data

## 2.1 Descriptions<a class="anchor" id="EXTRACT"></a>

**Utgård - previous measurements**

previous_1: SEQ.I background measurement, horizontal, unshielded
> e-log: https://stf02.nuclear.lu.se/MG.CSPEC/106

previous_2: SEQ.I, horizontal, shielded inside mirrobor+polyethylene cave
> e-log: https://stf02.nuclear.lu.se/MG.CSPEC/107

previous_3: SEQ.I, vertical, unshielded. Two detectors.
> e-log: https://stf02.nuclear.lu.se/MG.CSPEC/108

previous_4: SEQ.I & SEQ.II, horizontal, unshielded
> e-log: https://stf02.nuclear.lu.se/MG.CSPEC/109

previous_5: SEQ.I & SEQ.II, horizontal, unshielded
> e-log: https://stf02.nuclear.lu.se/MG.CSPEC/110

previous_6: SEQ.I, horizontal, Mirrorbor shielding front and back
> e-log: https://stf02.nuclear.lu.se/MG.CSPEC/111

previous_7: SEQ.II, horizontal, Mirrorbor+polyethylene shielding front and back
> e-log: https://stf02.nuclear.lu.se/MG.CSPEC/112

previous_8: SEQ.I, horizontal, inside new cave
> e-log: https://stf02.nuclear.lu.se/MG.CSPEC/113



**Utgård - preparation measurements**

utgard_run_1: SEQ.I & SEQ.II, horizontal, mirrobor shielding front + sides
> e-log: https://stf02.nuclear.lu.se/MG.CSPEC/103

utgard_run_2: SEQ.I & SEQ.II, horizontal, mirrobor shielding front + sides
> e-log: https://stf02.nuclear.lu.se/MG.CSPEC/104

utgard_run_3: SEQ.I & SEQ.II, horizontal, no shielding
> e-log: https://stf02.nuclear.lu.se/MG.CSPEC/105

utgard_run_4: SEQ.I & SEQ.II, horizontal, mirrobor shielding with opening in the middle
> e-log: https://stf02.nuclear.lu.se/MG.CSPEC/115

utgard_run_5: SEQ.I & SEQ.II, horizontal, mirrobor shielding with opening at the top
> e-log: https://stf02.nuclear.lu.se/MG.CSPEC/116

utgard_run_6: SEQ.I & SEQ.II, horizontal, no shielding, test with T0-signal
> e-log: 

**LET - preparation measurements**

let_test_run_1: 
> e-log:

**LET - beam time**

let_beam_run_1:
> e-log:


## 2.2 Paths<a class="anchor" id="EXTRACT"></a>

In [ ]:
''' Declare paths to data '''
# UTGÅRD - PREVIOUS MEASUREMENTS
PREVIOUS_PATHS = {'previous_run_1': RAW_FOLDER + 'mvmelst_042_noShielding_68h.zip',
                  'previous_run_2': RAW_FOLDER + 'mvmelst_045_Shielding_1100V_114h22m.zip',
                  'previous_run_3': RAW_FOLDER + 'mvmelst_078_2det_45h56m_1vertical.zip',
                  
                  'previous_run_4': RAW_FOLDER + 'mvmelst_171_191018_130129_First_SEQUOIA_II_Meauserement.zip',
                  'previous_run_5': RAW_FOLDER + 'mvmelst_179_191118_065800_SEQUOIA_II_18_11_2019_to_20_11_2019_no_mirrorbor_shielding.zip',
                  'previous_run_6': RAW_FOLDER + 'mvmelst_181_191120_100402_SEQUOIA__II_20_11_2019_to_22_11_2019_mirrorbor_shielding_top_and_bottom.zip',
                  'previous_run_7': RAW_FOLDER + 'mvmelst_188_191203_101632_SEQUOIA_II_1350_V_03_12_2019_to_05_12_2019_SEQUOIA_II_shielded_with_polyethylene_and_mirrorbor_sides_and_front.zip',
                  'previous_run_8': RAW_FOLDER + 'mvmelst_189_191205_093810_SEQUOIA_I_06_12_2019_to_09_12_2019_inside_new_cave.zip'
                  }      

# UTGÅRD - PREPARATION MEASUREMENTS
UTGARD_PATHS = {'utgard_run_1': RAW_FOLDER + 'mvmelst_191_200305_111506_utgard_first_run.zip',
                'utgard_run_2': RAW_FOLDER + 'mvmelst_193_200306_040827_mirrobor_covering_front_and_sides.zip',
                'utgard_run_3': RAW_FOLDER + 'mvmelst_194_200307_102939_unshielded.zip',
                'utgard_run_4': RAW_FOLDER + 'mvmelst_195_200309_033822_mirrorbor_shielding_opening_in_middle.zip',
                'utgard_run_5': RAW_FOLDER + 'mvmelst_196_200311_121940_mirrorbor_shielding_opening_at_top.zip',
                'utgard_run_6': RAW_FOLDER + 'mvmelst_197_200312_045631_unshielded_first_t0_test.zip',
                'utgard_run_7': RAW_FOLDER + 'mvmelst_198_200312_075113_unshielded_second_t0_test.zip',
                'utgard_run_8': RAW_FOLDER + 'mvmelst_199_200312_084636_unshielded_no_t0_signals_sent.zip'
               }

# UTGÅRD - ANTON BACKGROUND MEASUREMENTS
ANTON_PATHS = {'anton_run_1': RAW_FOLDER + 'anton/' + 'mvmelst_030_Shielding_22h.zip',
               'anton_run_2': RAW_FOLDER + 'anton/' + 'mvmelst_031_Shielding_1100V_28h.zip',
               'anton_run_3': RAW_FOLDER + 'anton/' + 'mvmelst_032_Shielding_1100V_20h33m.zip',
               'anton_run_4': RAW_FOLDER + 'anton/' + 'mvmelst_033_Shielding_1100V_1h20m.zip',
               'anton_run_5': RAW_FOLDER + 'anton/' + 'mvmelst_034_NoShielding_1100V_70h17m.zip',
               'anton_run_6': RAW_FOLDER + 'anton/' + 'mvmelst_035_Shielding_1100V_69h20m.zip',
               'anton_run_7': RAW_FOLDER + 'anton/' + 'mvmelst_036_Shielding_1100V_28h.zip',
               'anton_run_8': RAW_FOLDER + 'anton/' + 'mvmelst_037_Shielding_1100V_69h09m.zip',
               'anton_run_9': RAW_FOLDER + 'anton/' + 'mvmelst_042_noShielding_68h.zip',
               'anton_run_10': RAW_FOLDER + 'anton/' + 'mvmelst_042_NoShileding_1100V_286h49m.zip',
               'anton_run_11': RAW_FOLDER + 'anton/' + 'mvmelst_043_Shielding_20h15m.zip',
               'anton_run_12': RAW_FOLDER + 'anton/' + 'mvmelst_043_Shielding_1100V_49h18m.zip',
               'anton_run_13': RAW_FOLDER + 'anton/' + 'mvmelst_045_Shielding_1100V_114h22m.zip',
               'anton_run_14': RAW_FOLDER + 'anton/' + 'mvmelst_047_NoShielding_Upright_1100V_268h16m.zip',
               'anton_run_15': RAW_FOLDER + 'anton/' + 'mvmelst_091_3det_69h34m.zip',
               'anton_run_16': RAW_FOLDER + 'anton/' + 'mvmelst_078_2det_45h56m_1vertical.zip',
               }      

## 2.3 Extract<a class="anchor" id="EXTRACT"></a>

In [ ]:
def extract_mg_data(zipped_path, clusters_save_path, events_save_path):
    unzipped_path = mg_read.unzip_data(zipped_path)
    data = mg_read.import_data(unzipped_path)
    # Extract clusters and save to disc
    clusters = mg_read.extract_clusters(data)
    mg_read.save_data(clusters, clusters_save_path)
    clusters = None
    # Extract events and save to disc
    events = mg_read.extract_events(data)
    mg_read.save_data(events, events_save_path)
    events = None
    # Clear data
    data = None

In [ ]:
# Extract data, cluster, and save to file
def extract_and_save(run, raw_path):
    clusters_path = PROCESSED_FOLDER + run + '_clu.h5'
    events_path = PROCESSED_FOLDER + run + '_ev.h5'
    extract_mg_data(raw_path, clusters_path, events_path)

### Extract data

In [ ]:
# UTGÅRD - PREVIOUS MEASUREMENTS
#extract_and_save('previous_run_1', PREVIOUS_PATHS['previous_run_1'])
#extract_and_save('previous_run_2', PREVIOUS_PATHS['previous_run_2'])
#extract_and_save('previous_run_3', PREVIOUS_PATHS['previous_run_3'])
#extract_and_save('previous_run_4', PREVIOUS_PATHS['previous_run_4'])
#extract_and_save('previous_run_5', PREVIOUS_PATHS['previous_run_5'])
#extract_and_save('previous_run_6', PREVIOUS_PATHS['previous_run_6'])
#extract_and_save('previous_run_7', PREVIOUS_PATHS['previous_run_7'])
#extract_and_save('previous_run_8', PREVIOUS_PATHS['previous_run_8'])

# UTGÅRD - PREPARATION MEASUREMENTS
#extract_and_save('utgard_run_1', UTGARD_PATHS['utgard_run_1'])
#extract_and_save('utgard_run_2', UTGARD_PATHS['utgard_run_2'])
#extract_and_save('utgard_run_3', UTGARD_PATHS['utgard_run_3'])
#extract_and_save('utgard_run_4', UTGARD_PATHS['utgard_run_4'])
#extract_and_save('utgard_run_5', UTGARD_PATHS['utgard_run_5'])
#extract_and_save('utgard_run_6', UTGARD_PATHS['utgard_run_6'])
#extract_and_save('utgard_run_7', UTGARD_PATHS['utgard_run_7'])
extract_and_save('utgard_run_8', UTGARD_PATHS['utgard_run_8'])

# UTGÅRD - ANTONS MEASUREMENTS
#extract_and_save('anton_run_1', ANTON_PATHS['anton_run_1'])
#extract_and_save('anton_run_2', ANTON_PATHS['anton_run_2'])
#extract_and_save('anton_run_3', ANTON_PATHS['anton_run_3'])
#extract_and_save('anton_run_4', ANTON_PATHS['anton_run_4'])
#extract_and_save('anton_run_5', ANTON_PATHS['anton_run_5'])
#extract_and_save('anton_run_6', ANTON_PATHS['anton_run_6'])
#extract_and_save('anton_run_7', ANTON_PATHS['anton_run_7'])
#extract_and_save('anton_run_8', ANTON_PATHS['anton_run_8'])
#extract_and_save('anton_run_9', ANTON_PATHS['anton_run_9'])
#extract_and_save('anton_run_10', ANTON_PATHS['anton_run_10'])
#extract_and_save('anton_run_11', ANTON_PATHS['anton_run_11'])
#extract_and_save('anton_run_12', ANTON_PATHS['anton_run_12'])
#extract_and_save('anton_run_13', ANTON_PATHS['anton_run_13'])
#extract_and_save('anton_run_14', ANTON_PATHS['anton_run_14'])
#extract_and_save('anton_run_15', ANTON_PATHS['anton_run_15'])
#extract_and_save('anton_run_16', ANTON_PATHS['anton_run_16'])

## 2.4 Load<a class="anchor" id="LOAD"></a>

In [ ]:
# Load clusters and events
def load_clusters_and_events(run):
    clusters_path = PROCESSED_FOLDER + run + '_clu.h5'
    events_path = PROCESSED_FOLDER + run + '_ev.h5'
    return mg_read.load_data(clusters_path), mg_read.load_data(events_path)

### Load data

In [ ]:
# UTGÅRD - PREVIOUS MEASUREMENTS
clu_previous_run_1, ev_previous_run_1 = load_clusters_and_events('previous_run_1')
clu_previous_run_2, ev_previous_run_2 = load_clusters_and_events('previous_run_2')
clu_previous_run_3, ev_previous_run_3 = load_clusters_and_events('previous_run_3')
clu_previous_run_4, ev_previous_run_4 = load_clusters_and_events('previous_run_4')
clu_previous_run_5, ev_previous_run_5 = load_clusters_and_events('previous_run_5')
clu_previous_run_6, ev_previous_run_6 = load_clusters_and_events('previous_run_6')
clu_previous_run_7, ev_previous_run_7 = load_clusters_and_events('previous_run_7')
clu_previous_run_8, ev_previous_run_8 = load_clusters_and_events('previous_run_8')

# UTGÅRD - PREPARATION MEASUREMENTs
clu_utgard_run_1, ev_utgard_run_1 = load_clusters_and_events('utgard_run_1')
clu_utgard_run_2, ev_utgard_run_2 = load_clusters_and_events('utgard_run_2')
clu_utgard_run_3, ev_utgard_run_3 = load_clusters_and_events('utgard_run_3')
clu_utgard_run_4, ev_utgard_run_4 = load_clusters_and_events('utgard_run_4')
clu_utgard_run_5, ev_utgard_run_5 = load_clusters_and_events('utgard_run_5')
clu_utgard_run_6, ev_utgard_run_6 = load_clusters_and_events('utgard_run_6')
clu_utgard_run_7, ev_utgard_run_7 = load_clusters_and_events('utgard_run_7')
clu_utgard_run_8, ev_utgard_run_8 = load_clusters_and_events('utgard_run_8')

# UTGÅRD - ANTON MEASUREMENTS
clu_anton_run_1, ev_anton_run_1 = load_clusters_and_events('anton_run_1')
clu_anton_run_2, ev_anton_run_2 = load_clusters_and_events('anton_run_2')
clu_anton_run_3, ev_anton_run_3 = load_clusters_and_events('anton_run_3')
clu_anton_run_4, ev_anton_run_4 = load_clusters_and_events('anton_run_4')
clu_anton_run_5, ev_anton_run_5 = load_clusters_and_events('anton_run_5')
clu_anton_run_6, ev_anton_run_6 = load_clusters_and_events('anton_run_6')
clu_anton_run_7, ev_anton_run_7 = load_clusters_and_events('anton_run_7')
clu_anton_run_8, ev_anton_run_8 = load_clusters_and_events('anton_run_8')
clu_anton_run_9, ev_anton_run_9 = load_clusters_and_events('anton_run_9')
clu_anton_run_10, ev_anton_run_10 = load_clusters_and_events('anton_run_10')
clu_anton_run_11, ev_anton_run_11 = load_clusters_and_events('anton_run_11')
clu_anton_run_12, ev_anton_run_12 = load_clusters_and_events('anton_run_12')
clu_anton_run_13, ev_anton_run_13 = load_clusters_and_events('anton_run_13')
clu_anton_run_14, ev_anton_run_14 = load_clusters_and_events('anton_run_14')
clu_anton_run_15, ev_anton_run_15 = load_clusters_and_events('anton_run_15')
clu_anton_run_16, ev_anton_run_16 = load_clusters_and_events('anton_run_16')

## 2.5 Filters<a class="anchor" id="FILTER"></a>

In [ ]:
"""
Filters are declared in the following format:
{'PARAMETER': [MIN_VALUE, MAX_VALUE, IS_ACTIVATE]}
"""

# Declare filter for multi-grid clusters
mg_basic_filter = {'wm': [1, 1, True],                   # Wire multiplicity
                   'gm': [1, 5, True],                   # Grid multiplicity
                   'wadc': [600, np.inf, True],          # Wire charge
                   'gadc': [600, np.inf, True],          # Grid charge
                   'tof': [0, np.inf, True],             # Time-of-flight (TDC channels)
                   'time': [0, np.inf, True],            # Time (TDC channels)
                   'bus': [0, 8, True],                  # Bus
                   'flag': [0, 1, False],                # =1 if different buses within same coincidence
                   'layer': [0, 19, False],              # Layer, front=0 to back=19
                   'row': [0, 11, False],                # Row, right to left (seen from neutrons)
                   'gch': [80, 119, True]}               # Grid channel, bottom=80 to top=119

# Declare filter for multi-grid clusters
mg_low_adc_filter = {'wm': [1, 1, True],                   # Wire multiplicity
                   'gm': [1, 5, True],                   # Grid multiplicity
                   'wadc': [0, np.inf, True],          # Wire charge
                   'gadc': [0, np.inf, True],          # Grid charge
                   'tof': [0, np.inf, True],             # Time-of-flight (TDC channels)
                   'time': [0, np.inf, True],            # Time (TDC channels)
                   'bus': [0, 8, True],                  # Bus
                   'flag': [0, 1, False],                # =1 if different buses within same coincidence
                   'layer': [0, 19, False],              # Layer, front=0 to back=19
                   'row': [0, 11, False],                # Row, right to left (seen from neutrons)
                   'gch': [80, 119, True]}               # Grid channel, bottom=80 to top=119

# Declare filter for multi-grid clusters
mg_high_adc_filter = {'wm': [1, 1, True],                   # Wire multiplicity
                   'gm': [1, 5, True],                   # Grid multiplicity
                   'wadc': [800, np.inf, True],          # Wire charge
                   'gadc': [800, np.inf, True],          # Grid charge
                   'tof': [0, np.inf, True],             # Time-of-flight (TDC channels)
                   'time': [0, np.inf, True],            # Time (TDC channels)
                   'bus': [0, 8, True],                  # Bus
                   'flag': [0, 1, False],                # =1 if different buses within same coincidence
                   'layer': [0, 19, False],              # Layer, front=0 to back=19
                   'row': [0, 11, False],                # Row, right to left (seen from neutrons)
                   'gch': [80, 119, True]}               # Grid channel, bottom=80 to top=119

# Declare filter for multi-grid clusters
mg_filter_no_edges = {'wm': [1, 1, True],                   # Wire multiplicity
                   'gm': [1, 5, True],                   # Grid multiplicity
                   'wadc': [600, np.inf, True],          # Wire charge
                   'gadc': [600, np.inf, True],          # Grid charge
                   'tof': [0, np.inf, True],             # Time-of-flight (TDC channels)
                   'time': [0, np.inf, True],            # Time (TDC channels)
                   'bus': [0, 8, True],                  # Bus
                   'flag': [0, 1, False],                # =1 if different buses within same coincidence
                   'layer': [0, 19, False],              # Layer, front=0 to back=19
                   'row': [0, 11, False],                # Row, right to left (seen from neutrons)
                   'gch': [82, 117, True]}               # Grid channel, bottom=80 to top=119

## 2.6 Plotting<a class="anchor" id="FILTER"></a>

In [ ]:
def plot_basic(run, clusters_unfiltered, events, df_filter, bus_start, bus_stop, number_rows, area):
    mg_hf.set_thick_labels(12)
    
    # Filter clusters
    clusters = mg_read.filter_data(clusters_unfiltered, df_filter)
    
    # Declare parameters
    duration = (clusters.time.values[-1] - clusters.time.values[0]) * 62.5e-9
    
    # PHS - 1D
    vmin = None
    vmax = None
    bins_phs_1d = 100
    fig = plt.figure()
    fig.set_figwidth(10)
    fig.set_figheight(3*number_rows)
    for bus in np.arange(bus_start, bus_stop+1, 1):
        plt.subplot(number_rows, 3, bus+1)
        events_bus = events[events.bus == bus]
        mg_basic_plot.phs_1d_plot(events_bus, bins_phs_1d, bus, vmin, vmax)
        #plt.yscale('log')
    plt.tight_layout()
    fig.show()
    
    # Save data
    #output_path = '../output/%s_phs_1d.png' % run
    #fig.savefig(output_path, bbox_inches='tight')
    
    # PHS - 2D
    vmin = 1
    vmax = events.shape[0] // 1000 + 100
    fig = plt.figure()
    fig.set_figwidth(10)
    fig.set_figheight(3*number_rows)
    for bus in np.arange(bus_start, bus_stop+1, 1):
        plt.subplot(number_rows, 3, bus+1)
        events_bus = events[events.bus == bus]
        if events_bus.shape[0] > 0:
            mg_basic_plot.phs_2d_plot(events_bus, bus, vmin, vmax)
    plt.tight_layout()
    fig.show()
    
    # Save data
    #output_path = '../output/%s_phs_2d.png' % run
    #fig.savefig(output_path, bbox_inches='tight')
    
    # Coincidences - 2D
    fig = plt.figure()
    fig.set_figwidth(10)
    fig.set_figheight(3*number_rows)
    if clusters.shape[0] != 0:
        vmin = 1
        vmax = clusters.shape[0] // 450 + 5
    else:
        duration = 1
        vmin = 1
        vmax = 1
    for bus in np.arange(bus_start, bus_stop+1, 1):
        plt.subplot(number_rows, 3, bus+1)
        clusters_bus = clusters[clusters.bus == bus]
        # Calculate number of events and rate in a specific bus
        number_events = clusters_bus.shape[0]
        events_per_s = number_events/duration
        events_per_s_m2 = events_per_s/area
        title = ('Bus %d\n(%d events, %.6f events/s/m$^2$)' % (bus, number_events, events_per_s_m2))
        if number_events > 0:
            mg_basic_plot.clusters_2d_plot(clusters_bus, title, vmin, vmax)
    plt.tight_layout()
    fig.show()
    
    # Save data
    #output_path = '../output/%s_coincidences_2d.png' % run
    #fig.savefig(output_path, bbox_inches='tight')
    
    # Coincidences - PHS
    fig = plt.figure()
    fig.set_figwidth(10)
    fig.set_figheight(3*number_rows)
    if clusters.shape[0] != 0:
        vmin = 1
        vmax = clusters.shape[0] // 450 + 1000
    else:
        duration = 1
        vmin = 1
        vmax = 1
    for bus in np.arange(bus_start, bus_stop+1, 1):
        plt.subplot(number_rows, 3, bus+1)
        clusters_bus = clusters[clusters.bus == bus]
        if clusters_bus.shape[0] > 0:
            mg_basic_plot.clusters_phs_plot(clusters_bus, bus, vmin, vmax)
    plt.tight_layout()
    fig.show()
    
    # Save data
    #output_path = '../output/%s_coincidences_phs.png' % run
    #fig.savefig(output_path, bbox_inches='tight')
    
    # Rate 
    number_bins = 50
    fig = plt.figure()
    fig.set_figwidth(10)
    fig.set_figheight(3*number_rows)
    for bus in np.arange(bus_start, bus_stop+1, 1):
        plt.subplot(number_rows, 3, bus+1)
        clusters_bus = clusters[clusters.bus == bus]
        mg_basic_plot.rate_plot(clusters_bus, number_bins, bus, area)
    plt.tight_layout()
    fig.show()
    
    # Save data
    #output_path = '../output/%s_rate.png' % run
    #fig.savefig(output_path, bbox_inches='tight')
    
    
    # TIME-OF-FLIGHT
    number_bins = 300
    fig = plt.figure()
    fig.set_figwidth(10)
    fig.set_figheight(3*number_rows)
    for bus in np.arange(bus_start, bus_stop+1, 1):
        plt.subplot(number_rows, 3, bus+1)
        clusters_bus = clusters[clusters.bus == bus]
        mg_basic_plot.tof_histogram(clusters_bus, number_bins, bus)
    plt.tight_layout()
    fig.show()

# 3. Utgård - previous measurements

## previous_run_1 (Anton, SEQ.I, unshielded, horizontal)

In [ ]:
area = 0.0225*0.0225*4*40
plot_basic('previous_run_1', clu_previous_run_1, ev_previous_run_1, mg_filter_no_edges, 0, 2, 1, area)

## previous_run_2 (Anton, SEQ.I, inside cave, horizontal)

In [ ]:
area = 0.0225*0.0225*4*40
plot_basic('previous_run_2', clu_previous_run_2, ev_previous_run_2, mg_filter_no_edges, 0, 2, 1, area)

## previous_run_3 (Anton, SEQ.I, unshielded, vertical)

In [ ]:
area = 0.0225*0.0225*4*40
plot_basic('previous_run_3', clu_previous_run_3, ev_previous_run_3, mg_basic_filter, 0, 5, 2, area)

## previous_run_4 (SEQ.II, unshielded, horizontal, first run)

In [ ]:
area = 0.0225*0.0225*4*40
plot_basic('previous_run_4', clu_previous_run_4, ev_previous_run_4, mg_basic_filter, 0, 5, 2, area)

## previous_run_5 (SEQ.I, unshielded, horizontal)

In [ ]:
area = 0.0225*0.0225*4*40
plot_basic('previous_run_5', clu_previous_run_5, ev_previous_run_5, mg_basic_filter, 0, 5, 2, area)

## previous_run_6 (SEQ.I, mirrobor shielding, horizontal)

In [ ]:
area = 0.0225*0.0225*4*40
plot_basic('previous_run_6', clu_previous_run_6, ev_previous_run_6, mg_basic_filter, 0, 5, 2, area)

## previous_run_7 (SEQ.II, mirrorbor+polyethylene, horizontal)

In [ ]:
area = 0.0225*0.0225*4*40
plot_basic('previous_run_7', clu_previous_run_7, ev_previous_run_7, mg_basic_filter, 0, 5, 2, area)

## previous_run_8 (SEQ.I, new cave, horizontal)

In [ ]:
area = 0.0225*0.0225*4*36
plot_basic('previous_run_8', clu_previous_run_8, ev_previous_run_8, mg_filter_no_edges, 0, 5, 2, area)

# 4. Utgård - preparations measurements

## utgard_run_1 (SEQ.I and SEQ.II, shielded with Mirrobor, horizontal)

In [ ]:
%matplotlib notebook

In [ ]:
area = 0.0225*0.0225*4*40
plot_basic('utgard_run_1', clu_utgard_run_1, ev_utgard_run_1, mg_high_adc_filter_no_edges, 0, 5, 2, area)

## utgard_run_2 (SEQ.I and SEQ.II, shielded with Mirrobor, horizontal)

In [ ]:
area = 0.0225*0.0225*4*36
plot_basic('utgard_run_2', clu_utgard_run_2, ev_utgard_run_2, mg_filter_no_edges, 0, 5, 2, area)

## utgard_run_3 (SEQ.I and SEQ.II, unshielded, horizontal)

In [ ]:
area = 0.0225*0.0225*4*36
plot_basic('utgard_run_3', clu_utgard_run_3, ev_utgard_run_3, mg_filter_no_edges, 0, 5, 2, area)

## utgard_run_4 (SEQ.I and SEQ.II, shielded with opening in middle, horizontal)

In [ ]:
area = 0.0225*0.0225*4*40
plot_basic('utgard_run_4', clu_utgard_run_4, ev_utgard_run_4, mg_high_adc_filter, 0, 5, 2, area)

## utgard_run_5 (SEQ.I and SEQ.II, shielded with opening at top, horizontal)

In [ ]:
area = 0.0225*0.0225*4*40
plot_basic('utgard_run_5', clu_utgard_run_5, ev_utgard_run_5, mg_high_adc_filter, 0, 5, 2, area)

In [ ]:
area = 0.0225*0.0225*4*40
plot_basic('utgard_run_6', clu_utgard_run_6, ev_utgard_run_6, mg_high_adc_filter, 0, 5, 2, area)

In [ ]:
area = 0.0225*0.0225*4*40
plot_basic('utgard_run_7', clu_utgard_run_7, ev_utgard_run_7, mg_filter_no_edges, 0, 5, 2, area)

In [ ]:
area = 0.0225*0.0225*4*36
plot_basic('utgard_run_8', clu_utgard_run_8, ev_utgard_run_8, mg_filter_no_edges, 0, 5, 2, area)
print(clu_utgard_run_8)

# 5. Utgård - Antons measurements

In [ ]:
%matplotlib notebook
area = 0.0225*0.0225*4*36
plot_basic('anton_run_1', clu_anton_run_1, ev_anton_run_1, mg_filter_no_edges, 0, 2, 1, area)

In [ ]:
area = 0.0225*0.0225*4*36
plot_basic('anton_run_2', clu_anton_run_2, ev_anton_run_2, mg_filter_no_edges, 0, 2, 1, area)

In [ ]:
area = 0.0225*0.0225*4*36
plot_basic('anton_run_3', clu_anton_run_3, ev_anton_run_3, mg_filter_no_edges, 0, 2, 1, area)

In [ ]:
area = 0.0225*0.0225*4*36
plot_basic('anton_run_4', clu_anton_run_4, ev_anton_run_4, mg_filter_no_edges, 0, 2, 1, area)

In [ ]:
area = 0.0225*0.0225*4*36
plot_basic('anton_run_5', clu_anton_run_5, ev_anton_run_5, mg_filter_no_edges, 0, 2, 1, area)

In [ ]:
area = 0.0225*0.0225*4*36
plot_basic('anton_run_6', clu_anton_run_6, ev_anton_run_6, mg_filter_no_edges, 0, 2, 1, area)

In [ ]:
area = 0.0225*0.0225*4*36
plot_basic('anton_run_7', clu_anton_run_7, ev_anton_run_7, mg_filter_no_edges, 0, 2, 1, area)

In [ ]:
area = 0.0225*0.0225*4*36
plot_basic('anton_run_8', clu_anton_run_8, ev_anton_run_8, mg_filter_no_edges, 0, 2, 1, area)

In [ ]:
area = 0.0225*0.0225*4*36
plot_basic('anton_run_9', clu_anton_run_9, ev_anton_run_9, mg_filter_no_edges, 0, 2, 1, area)

In [ ]:
area = 0.0225*0.0225*4*36
plot_basic('anton_run_10', clu_anton_run_10, ev_anton_run_10, mg_filter_no_edges, 0, 2, 1, area)

In [ ]:
area = 0.0225*0.0225*4*36
plot_basic('anton_run_11', clu_anton_run_11, ev_anton_run_11, mg_filter_no_edges, 0, 2, 1, area)

In [ ]:
area = 0.0225*0.0225*4*36
plot_basic('anton_run_12', clu_anton_run_12, ev_anton_run_12, mg_filter_no_edges, 0, 2, 1, area)

In [ ]:
area = 0.0225*0.0225*4*36
plot_basic('anton_run_13', clu_anton_run_13, ev_anton_run_13, mg_filter_no_edges, 0, 2, 1, area)

In [ ]:
area = 0.0225*0.0225*4*36
plot_basic('anton_run_14', clu_anton_run_14, ev_anton_run_14, mg_filter_no_edges, 0, 2, 1, area)

In [ ]:
area = 0.0225*0.0225*4*36
#plot_basic('anton_run_15', clu_anton_run_15, ev_anton_run_15, mg_filter_no_edges, 0, 8, 3, area)

In [ ]:
area = 0.0225*0.0225*4*36
plot_basic('anton_run_16', clu_anton_run_16, ev_anton_run_16, mg_filter_no_edges, 0, 5, 2, area)